# Main config file builder

*Februari 2017, Kiefer van Teutem (kiefert@nikhef.nl), Erik Hogenbirk (ehogenbi@nikhef.nl, hogenbirk91@gmail.com)*

This script is intended to output configuration files for kodiaq, which is controlled by the daqcontrol. Find repository [here](https://github.com/ErikHogenbirk/XAMS).

### Imports, settings, functions

In [55]:
# Imports
import json
import time
import os
from copy import deepcopy

In [56]:
# Configuration of the configuration maker
output_dir = '/home/xams/xams/daqcontrol/ini/'
filename = 'default_config.ini'

In [57]:
def set_register(cf, reg, val, comment="", verbose=False):
    '''This sets one register value of a config dict.'''
    # This is a list of dicts
    reg_list = cf['registers']
    new_reg_list = []
    reg_found = False
    for reg_dict in reg_list:
        # Find the dict with that register
        if reg_dict['register'] == reg:
            # If we found it, let's change it
            reg_dict['value'] = val
            if verbose: print("Just set register %s (%s) with value %s" % (reg_dict['register'],
                                                                          reg_dict['comment'], reg_dict['value']))
            # Keep track of if we found it
            reg_found = True
            new_reg_list.append(reg_dict)
        else:
            # If it doesn't match, we must just copy it
            new_reg_list.append(reg_dict)
    if not reg_found:
        # This is a NEW register, so we must build it
        print("Register %s not found in config, will append it now..." % reg)
        reg_dict = {
            "comment" : comment,
            "register": reg,
            "board" : "-1",
            "value" : val
        }
        new_reg_list.append(reg_dict)
    cf['registers'] = new_reg_list
    return cf
        

In [58]:
def save_config(cf, filename, output_dir=output_dir):
    config_path = os.path.join(output_dir, filename)
    with open(config_path, 'w') as outfile:
        json.dump(cf, outfile, indent = 4)
    print('Output written to %s' % config_path)

### Default

In [59]:
cf_default = {
    "data_folder" : "/home/xams/lena/run8/raw/data",
    "lena_pax_config_path" : "/data/xenon/xams/run8/pax_ini/XAMS_raw_to_processed.ini",
    "zle" : False,
    "links": [{
        "type": "V2718",
        "reader": 0,
        "crate": 0,
        "link": 0
    }],

    "boards": [{
        "crate": 0,
        "serial": "077",
        "reader": 0,
        "type": "V1724",
        "vme_address": "Whatever?",
        "link": 0
    }],
    # blt_size {int} : block transfer size in bytes (decimal).
    # V1724 standard: 524288
    # V1730 maximum: 2097152?
    "blt_size": 524288,
    # run_start {int} : 0 - board internal (run start via register), 
    # 1 - s-in (for synchronized boards)
    "run_start": 0,
    "baseline_mode": 0,
    # write_mode {int}: 0-no writing, 1-to file, 2-mongodb
    "write_mode": 2,

    "mongo": {
        "write_concern": 0,
        "min_insert_size": 1,
        "database": "xamsdata0",
        "address": "mongodb://localhost:27017",
        "collection": "onzin"
    },
    "processing_readout_threshold": 0,
    "processing_num_threads": 8,
    "processing_mode": 3,
    "occurrence_integral": 0,

    "compression": 1,
    
    "registers": [
        {
            "comment": "Reset software (all registers)",
            "register": "EF24",
            "board": "-1",
            "value": "1"
        },
        {
            "comment": "BLT event number",
            "register": "EF1C",
            "board": "-1",
            "value": "40"
            # Maximum number of events loaded into each block.
            # 40hex = 64dec
        },
        {
            "comment": "Enable BERR (error messages)",
            "register": "EF00",
            "board": "-1",
            "value": "10"
        },
        {
            "comment": "Aquisition control register",
            "register": "8100",
            "board": "-1",
            "value": "5" 
            #5: s-in control, 0: board control
        },
        {
            "comment": "Channel calibration",
            "register": "809C",
            "board": "-1",
            "value": "1"
            # enables channel calibration
        },
        {
            "comment": "Channel enable mask",
            "register": "8120",
            "board": "-1",
            "value": "09"
            # set to 01 to enable channel 0 only
            # set to 09 to enable channel 0 and 3
        },
        {
            "comment": "Measurement time after trigger",
            "register": "8114",
            "board": "-1",
            "value": "1400"
            # 20hex = 32dec = 320 samples = 640 ns
            # 1000 = 81 920 ns
        },
        {
            "comment": "Trigger source enable mask",
            "register": "810C",
            "board": "-1",
            "value": "C1F00003"
            # First digit: C for enable external + software trig
            # Second: coincidence level
            # Last one (?): channels enable
            # Set to "C0000001" for only trigger 0 and 1
        },
        {
            "comment": "Buffer organisation",
            "register": "800C",
            "board": "-1",
            "value": "6"
        },
        {
            "comment": "Interrupt status ID",
            "register": "EF14",
            "board": "-1",
            "value": "55AA"
            # Interrupt
        },
        {
            "comment": "Interrupt event number",
            "register": "EF18",
            "board": "-1",
            "value": "1"
            # Interrupt when 1 event stored
        },
        {
            "comment": "Channel 0 gain",
            "register": "1028",
            "board": "-1",
            "value": "0"
            # Channel n gain 0x1n28 (either 0 (2 Vpp) or 1 (0.5 Vpp))
            # ONLY V1730
        },
        {
            "comment": "Channel 3 gain",
            "register": "1328",
            "board": "-1",
            "value": "0"
            # Channel n gain 0x1n28 (either 0 (2 Vpp) or 1 (0.5 Vpp))
            # ONLY V1730
        },
        {
            "comment": "Trigger level channel 0",
            "register": "1080",
            "board": "-1",
            "value": "1FD7"
            # 1FF4 (hex) = 8180 (dec)
            # 1FEA (hex) = 8170 (dec)
            # 1FE0 (hex) = 8160 (dec)
            # 1FD6 (hex) = 8150 (dec)
            # 1F40 (hex) = 8000 (dec)
            # 1E78 (hex) = 7800 (dec)
        },
        {
            "comment": "Trigger level channel 3",
            "register": "1380",
            "board": "-1",
            "value": "201B"
        },
        {
            "comment": "Trigger polarity",
            "register": "8000",
            "board": "-1",
            "value": "40"
            # Set to 40 for negative, 00 for positive
        }
    ],
}


In [60]:
save_config(cf_default, 'default_config.ini')

Output written to /home/xams/xams/daqcontrol/ini/default_config.ini


## Other configuration

### Default ZLE

In [61]:
cf_default_zle = deepcopy(cf_default)
cf_default_zle['zle'] = True
save_config(cf_default_zle, 'default_zle.ini')

Output written to /home/xams/xams/daqcontrol/ini/default_zle.ini


### Gain calibration

Settings for PMT calibration. Triggers on a NIM pulse fed to channel 4. Short waveforms (1.28 us) with the PMT pulses that around 700 ns.

In [62]:
cf_PMT12_gain = deepcopy(cf_default)
cf_PMT12_gain['data_folder'] = "/home/xams/lena/run8/raw/pmtdata"
cf_PMT12_gain["lena_pax_config_path"] = "/data/xenon/xams/run8/pax_ini/XAMS_raw_to_processed_PMT12_gain.ini"
cf_PMT12_gain = set_register(cf_PMT12_gain, "810C", "C0000004") # Trigger source
cf_PMT12_gain = set_register(cf_PMT12_gain, "8114", "20")       # Time after trigger
cf_PMT12_gain = set_register(cf_PMT12_gain, "800C", "A")        # Buffer organization
cf_PMT12_gain = set_register(cf_PMT12_gain, "1028", "1")        # Channel 0 gain
cf_PMT12_gain = set_register(cf_PMT12_gain, "1328", "1")        # Channel 3 gain
cf_PMT12_gain = set_register(cf_PMT12_gain, "1428", "0")        # Channel 4 gain
cf_PMT12_gain = set_register(cf_PMT12_gain, "1080", "0000")     # Trigger level channel 0
cf_PMT12_gain = set_register(cf_PMT12_gain, "1380", "0000")     # Trigger level channel 3
cf_PMT12_gain = set_register(cf_PMT12_gain, "1480", "1770")     # Trigger level channel 4 (at 6000 counts, for NIM pulse)
cf_PMT12_gain = set_register(cf_PMT12_gain, "8020", "40")       # Event window size (40hex = 64dec = 
                                                                #                 640 samples = 1280 ns)
save_config(cf_PMT12_gain, 'PMT12_gain.ini')

Register 1428 not found in config, will append it now...
Register 1480 not found in config, will append it now...
Register 8020 not found in config, will append it now...
Output written to /home/xams/xams/daqcontrol/ini/PMT12_gain.ini


### Baseline

Same general configuration, but with the exception that you can trigger on external trigger only. Useful for noise, baseline, on data rate test.

In [63]:
cf_baseline = deepcopy(cf_default)
cf_baseline['data_folder'] = "/home/xams/lena/run8/raw/baseline"
cf_baseline["lena_pax_config_path"] = "/data/xenon/xams/run8/pax_ini/XAMS_raw_to_processed_baseline.ini"
cf_baseline = set_register(cf_baseline, "810C", "C0000004")  # Trigger source: use ONLY external (or channel 4 0.5V pulse)
cf_baseline = set_register(cf_baseline, "1480", "1770")  # Trigger level channel 4 (-0.25 volt)
cf_baseline = set_register(cf_baseline, "1428", "0")  # Gain channel 4

save_config(cf_baseline, 'baseline.ini')

Register 1480 not found in config, will append it now...
Register 1428 not found in config, will append it now...
Output written to /home/xams/xams/daqcontrol/ini/baseline.ini


### Baseline ZLE

In [64]:
cf_baseline_zle = deepcopy(cf_baseline)
cf_baseline_zle['zle'] = True
save_config(cf_baseline_zle, 'baseline_zle.ini')

Output written to /home/xams/xams/daqcontrol/ini/baseline_zle.ini


### Radon detector

In [65]:
cf_rn = deepcopy(cf_default)
cf_rn['data_folder'] = '/home/xams/lena/run8/raw/radon_detector'
cf_rn['lena_pax_config_path'] = '/data/xenon/xams/run8/pax_ini/XAMS_radon_detector.ini'
cf_rn = set_register(cf_rn, "810C", "C0000004") # Trigger source: channels 4 and 5. 
                                                # Trigger on timing pulse
cf_rn = set_register(cf_rn, "1480", "1FD6")     # Trigger level channel 4
cf_rn = set_register(cf_rn, "8114", "20")       # Time after trigger
cf_rn = set_register(cf_rn, "800C", "A")        # Buffer organization
cf_rn = set_register(cf_rn, "8120", "11")       # Channel enable mask: Channels 0 and 4
cf_rn = set_register(cf_rn, "8020", "40")       # Event window size (40hex = 64dec = 
                                                #                 640 samples = 1280 ns)
cf_rn = set_register(cf_rn, "8000", '40')       # Trigger polarity
save_config(cf_rn, 'radon_detector.ini')

Register 1480 not found in config, will append it now...
Register 8020 not found in config, will append it now...
Output written to /home/xams/xams/daqcontrol/ini/radon_detector.ini


### Xenon_gas

In [66]:
cf_rn = deepcopy(cf_default)
cf_rn['data_folder'] = '/home/xams/lena/run8/raw/xenon_gas'
cf_rn['lena_pax_config_path'] = '/data/xenon/xams/run8/pax_ini/xenon_gas.ini'
cf_rn = set_register(cf_rn, "1080", "1FFB")     # Trigger level channel 0
cf_rn = set_register(cf_rn, "1380", "2021")     # Trigger level channel 3
cf_rn = set_register(cf_rn, "8114", "2000")     # Time after trigger
cf_rn = set_register(cf_rn, "800C", "5")        # Buffer organization
cf_rn = set_register(cf_rn, "EF1C", "20")       # 
cf_rn['blt_size'] = 2097152
save_config(cf_rn, 'xenon_gas.ini')

Output written to /home/xams/xams/daqcontrol/ini/xenon_gas.ini


### S1 only

In [67]:
cf_s1 = deepcopy(cf_default)
cf_s1['data_folder'] = '/home/xams/lena/run8/raw/s1_only'
cf_s1['lena_pax_config_path'] = '/data/xenon/xams/run8/pax_ini/s1_only.ini'
cf_s1 = set_register(cf_s1, "1080", "1FFB")     # Trigger level channel 0
cf_s1 = set_register(cf_s1, "1380", "2021")     # Trigger level channel 3
cf_s1 = set_register(cf_s1, "8114", "0100")     # Time after trigger
                                                # 100hex = 256dec = 2560 samples = 5120 ns
cf_s1 = set_register(cf_s1, "800C", "A")        # Buffer organization
                                                # A = 10 us
save_config(cf_s1, 's1_only.ini')

Output written to /home/xams/xams/daqcontrol/ini/s1_only.ini


In [68]:
def get_trigger_level(channel, counts):
    if channel==0:
        level = 8228 - counts
    
    if channel ==3:
        level = 8266 - counts
    return hex(level)

In [69]:
int('2000', 16) * 10 * 2

163840

In [70]:
8192*10*2

163840

### Na22_coincidence

In [71]:
cf_bla = deepcopy(cf_default)
cf_bla['zle'] = True

cf_bla['data_folder'] = "/home/xams/lena/run8/raw/Na22"
cf_bla["lena_pax_config_path"] = "/data/xenon/xams/run8/pax_ini/XAMS_raw_to_processed_Na22.ini"

cf_bla = set_register(cf_bla, "8120", "19")       # Channel enable mask
cf_bla = set_register(cf_bla, "810C", "C2F00007") # Trigger source
cf_bla = set_register(cf_bla, "1480", "2000")     # Trigger level channel 4

save_config(cf_bla, 'Na22_zle.ini')

Register 1480 not found in config, will append it now...
Output written to /home/xams/xams/daqcontrol/ini/Na22_zle.ini


### NaI_only

In [72]:
cf_bla = deepcopy(cf_default)
cf_bla['zle'] = False

cf_bla['data_folder'] = "/home/xams/lena/run8/raw/NaI_only"
cf_bla["lena_pax_config_path"] = "/data/xenon/xams/run8/pax_ini/XAMS_raw_to_processed_NaI_only.ini"

cf_bla = set_register(cf_bla, "8114", "60")       # Time after trigger
cf_bla = set_register(cf_bla, "800C", "A")        # Buffer organization
cf_bla = set_register(cf_bla, "8020", "C0")       # Event window size (40hex = 1280ns)
cf_bla = set_register(cf_bla, "8120", "10")       # Channel enable mask
cf_bla = set_register(cf_bla, "810C", "C0000004") # Trigger source
cf_bla = set_register(cf_bla, "1080", "0000")     # Trigger level channel 0
cf_bla = set_register(cf_bla, "1380", "0000")     # Trigger level channel 3
cf_bla = set_register(cf_bla, "1480", "2000")     # Trigger level channel 4

save_config(cf_bla, 'NaI_only.ini')

Register 8020 not found in config, will append it now...
Register 1480 not found in config, will append it now...
Output written to /home/xams/xams/daqcontrol/ini/NaI_only.ini
